In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rutujakawadeee","key":"289e254104e30385c6493e67b2c507d0"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d emmarex/plantdisease -p /content


Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease
License(s): unknown
 99% 653M/658M [00:12<00:00, 99.2MB/s]
100% 658M/658M [00:12<00:00, 53.3MB/s]


In [ ]:
!unzip -q /content/plantdisease.zip -d /content/plantvillage_raw


In [ ]:
!ls /content/plantvillage_raw/plantvillage/PlantVillage | head


Pepper__bell___Bacterial_spot
Pepper__bell___healthy
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Tomato_Bacterial_spot
Tomato_Early_blight
Tomato_healthy
Tomato_Late_blight
Tomato_Leaf_Mold


In [ ]:
!pip install split-folders


In [ ]:
import splitfolders

splitfolders.ratio(
    "/content/plantvillage_raw/plantvillage/PlantVillage",
    output="/content/plantvillage_split",
    seed=42,
    ratio=(.8, .2)
)

Copying files: 20639 files [00:05, 3574.15 files/s]


In [ ]:
!find /content/plantvillage_split/train -type f | wc -l
!find /content/plantvillage_split/val -type f | wc -l


16505
4134


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [ ]:
train_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
])

val_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


In [ ]:
train_dir = "/content/plantvillage_split/train"
val_dir   = "/content/plantvillage_split/val"

train_data = datasets.ImageFolder(train_dir, transform=train_tfms)
val_data   = datasets.ImageFolder(val_dir, transform=val_tfms)

print("Classes:", train_data.classes)
print("Train images:", len(train_data))
print("Val images:", len(val_data))


Classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']
Train images: 16504
Val images: 4134


In [ ]:
import os

BASE_DIR = "/content/drive/MyDrive/PlantDocBot"

folders = [
    "data/raw",
    "data/train",
    "data/val",
    "models",
    "notebooks",
    "app"
]

for f in folders:
    os.makedirs(os.path.join(BASE_DIR, f), exist_ok=True)

print("Project folders created successfully")


Project folders created successfully


In [ ]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d emmarex/plantdisease -p /content


Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease
License(s): unknown
plantdisease.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!ls /content/plantvillage_raw/plantvillage/PlantVillage | head


Pepper__bell___Bacterial_spot
Pepper__bell___healthy
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Tomato_Bacterial_spot
Tomato_Early_blight
Tomato_healthy
Tomato_Late_blight
Tomato_Leaf_Mold


In [ ]:
import os, shutil, random
from glob import glob

src = "/content/plantvillage_raw/plantvillage/PlantVillage"
dst = f"{BASE_DIR}/data"

train_dir = os.path.join(dst, "train")
val_dir = os.path.join(dst, "val")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)


In [ ]:
split_ratio = 0.8
classes = os.listdir(src)

for cls in classes:
    cls_path = os.path.join(src, cls)
    images = glob(cls_path + "/*.jpg")
    random.shuffle(images)

    split = int(len(images) * split_ratio)
    train_imgs = images[:split]
    val_imgs = images[split:]

    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

    for img in train_imgs:
        shutil.copy(img, os.path.join(train_dir, cls))

    for img in val_imgs:
        shutil.copy(img, os.path.join(val_dir, cls))

print("Train/Validation split completed")


Train/Validation split completed


In [ ]:
def clean_folders(path):
    for folder in os.listdir(path):
        old = os.path.join(path, folder)
        new = os.path.join(path, folder.lower().replace(" ", "_"))
        if old != new:
            os.rename(old, new)

clean_folders(train_dir)
clean_folders(val_dir)


In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader


In [ ]:
train_tfms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

val_tfms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])


In [ ]:
train_data = datasets.ImageFolder(train_dir, transform=train_tfms)
val_data = datasets.ImageFolder(val_dir, transform=val_tfms)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)


FileNotFoundError: Found no valid file for the classes pepper__bell___bacterial_spot, pepper__bell___healthy, potato___early_blight, potato___healthy, potato___late_blight, tomato__target_spot, tomato__tomato_mosaic_virus, tomato__tomato_yellowleaf__curl_virus, tomato_bacterial_spot, tomato_early_blight, tomato_healthy, tomato_leaf_mold, tomato_septoria_leaf_spot, tomato_spider_mites_two_spotted_spider_mite. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [ ]:
import os

def lowercase_images(root):
    for path, _, files in os.walk(root):
        for f in files:
            old = os.path.join(path, f)
            new = os.path.join(path, f.lower())
            if old != new:
                os.rename(old, new)

lowercase_images(train_dir)
lowercase_images(val_dir)

print("All image filenames converted to lowercase")


All image filenames converted to lowercase


In [ ]:
import shutil

def remove_empty_folders(root):
    for folder in os.listdir(root):
        full = os.path.join(root, folder)
        if os.path.isdir(full) and len(os.listdir(full)) == 0:
            print("Removing empty folder:", full)
            shutil.rmtree(full)

remove_empty_folders(train_dir)
remove_empty_folders(val_dir)


Removing empty folder: /content/drive/MyDrive/PlantDocBot/data/train/pepper__bell___bacterial_spot
Removing empty folder: /content/drive/MyDrive/PlantDocBot/data/train/tomato_bacterial_spot
Removing empty folder: /content/drive/MyDrive/PlantDocBot/data/train/tomato_leaf_mold
Removing empty folder: /content/drive/MyDrive/PlantDocBot/data/train/potato___healthy
Removing empty folder: /content/drive/MyDrive/PlantDocBot/data/train/tomato_spider_mites_two_spotted_spider_mite
Removing empty folder: /content/drive/MyDrive/PlantDocBot/data/train/tomato_early_blight
Removing empty folder: /content/drive/MyDrive/PlantDocBot/data/train/potato___late_blight
Removing empty folder: /content/drive/MyDrive/PlantDocBot/data/train/tomato_healthy
Removing empty folder: /content/drive/MyDrive/PlantDocBot/data/train/tomato__target_spot
Removing empty folder: /content/drive/MyDrive/PlantDocBot/data/train/potato___early_blight
Removing empty folder: /content/drive/MyDrive/PlantDocBot/data/train/tomato__tomat

In [ ]:
print("Train classes:", os.listdir(train_dir)[:5])
print("Val classes:", os.listdir(val_dir)[:5])

# Check one class
sample_class = os.listdir(train_dir)[0]
print("Sample images:", os.listdir(os.path.join(train_dir, sample_class))[:5])


Train classes: ['tomato_late_blight']
Val classes: ['tomato_healthy', 'tomato_late_blight', 'pepper__bell___healthy']
Sample images: ['de2e16e8-b372-4d2e-ab3b-30b21c30faa7___ghlb_ps leaf 1.5 day 18.jpg', '12125c80-42e3-4970-bb92-524262c337fa___ghlb_ps leaf 8.1 day 7.jpg', '6b7f4762-88d6-4e85-8fad-57ca4083ec89___ghlb_ps leaf 8.1 day 9.jpg', '1159051e-1278-4b35-929c-e26b963d048f___ghlb_ps leaf 8.1 day 16.jpg', 'a93a7ca0-3ed4-4d86-9441-49a6d605c26d___ghlb_ps leaf 26 day 9.jpg']


In [ ]:
from torchvision import datasets
from torch.utils.data import DataLoader

train_data = datasets.ImageFolder(train_dir, transform=train_tfms)
val_data   = datasets.ImageFolder(val_dir, transform=val_tfms)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=32)

print("Train images:", len(train_data))
print("Val images:", len(val_data))
print("Classes:", train_data.classes)


Train images: 121
Val images: 33
Classes: ['tomato_late_blight']


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(weights="IMAGENET1K_V1")
model.fc = nn.Linear(model.fc.in_features, len(train_data.classes))
model = model.to(device)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 58.1MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
for epoch in range(2):
    model.train()
    running_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {running_loss:.4f}")


Epoch 1 Loss: 0.0000
Epoch 2 Loss: 0.0000


In [ ]:
import os, shutil, random

raw_dir = "/content/plantvillage_data/plantvillage/PlantVillage"
train_dir = "/content/data/train"
val_dir = "/content/data/val"

split_ratio = 0.8

for cls in os.listdir(raw_dir):
    cls_path = os.path.join(raw_dir, cls)
    if not os.path.isdir(cls_path):
        continue

    images = os.listdir(cls_path)
    if len(images) == 0:
        continue

    random.shuffle(images)
    split = int(len(images) * split_ratio)

    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

    for img in images[:split]:
        shutil.copy(
            os.path.join(cls_path, img),
            os.path.join(train_dir, cls, img)
        )

    for img in images[split:]:
        shutil.copy(
            os.path.join(cls_path, img),
            os.path.join(val_dir, cls, img)
        )

print("✅ Dataset split completed")

FileNotFoundError: [Errno 2] No such file or directory: '/content/plantvillage_data/plantvillage/PlantVillage'